In [1]:
import numpy as np
import pandas as pd
from math import sqrt,acos,degrees
import matplotlib.pyplot as plt
from scipy.signal import resample
from tqdm.notebook import tqdm
from time import time
import os
import csv

In [4]:
def extract_dataframes_test(path,file):
    file_s = file
    file = '/marker_data/' + file
    
    with open(path+'/'+file, "r") as f:
        lines = f.readlines()
    emg_labels = ['Frame','Sub Frame',
                 'IM EMG1',
                 'IM EMG2',
                 'IM EMG3',
                 'IM EMG4',
                 'IM EMG5',
                 'IM EMG6',
                 'IM EMG7',
                 'IM EMG8',
                 'IM EMG9',
                 'IM EMG10',
                 'IM EMG11',
                 'IM EMG12']
    emg_labels_ref = ['Frame','Sub Frame',
                 'EMG1',
                 'EMG2',
                 'EMG3',
                 'EMG4',
                 'EMG5',
                 'EMG6',
                 'EMG7',
                 'EMG8',
                 'EMG9',
                 'EMG10',
                 'EMG11',
                 'EMG12']
    marker_labels = ['Frame','Sub Frame',
                  'RSHO_X','RSHO_Y','RSHO_Z',
                  'RUPA_X','RUPA_Y','RUPA_Z',
                  'RELB_X','RELB_Y','RELB_Z',
                  'RM1_X','RM1_Y','RM1_Z',
                 'RFRM_X','RFRM_Y','RFRM_Z',
                 'WRM2_X','WRM2_Y','WRM2_Z',
                 'RWRA_X','RWRA_Y','RWRA_Z',
                 'RWRB_X','RWRB_Y','RWRB_Z',
                 'RFIN_X','RFIN_Y','RFIN_Z']
    
    pronation_movement = ['No Motion','Supination','Pronation']
    flexion_movement = ['No Motion','Extension','Flexion']
    radial_movement = ['No Motion','Ulnar','Radial']
    dtm_movement = ['No Motion','Forward','Backward']
    
    #################
    # EMG Data Frame#
    #################
    
    emg_lines = []
    for line in lines[5:]:
        if line=='\n':
            break
        emg_lines.append(line.split(','))
    emg_df = pd.DataFrame(np.array(emg_lines),columns=lines[3].split(','))
    emg_df = emg_df[emg_labels]
    emg_df.columns = emg_labels_ref
#     print(emg_df)
#     print(emg_df.info())
#     print(emg_df.describe())
#     emg_df = emg_df.interpolate()
    emg_df = emg_df[emg_df.columns].astype(float)
    duration = emg_df.shape[0]/2000
    
    # Marker Data Frame
    marker_lines = []
    marker_line_start = None
    for i in range(len(lines)):
        if lines[i]=='Trajectories\n':
            marker_line_start = i
            break
    for line in lines[marker_line_start+5:]:
        if line=='\n':
            break
        marker_lines.append(line.split(','))
    marker_df = pd.DataFrame(np.array(marker_lines),columns=marker_labels)
    marker_df = marker_df[marker_df.columns].astype(float)
    marker_df = marker_df.interpolate()
        
    return marker_df

In [14]:
emg_labels = ['Frame','Sub Frame',
             'IM EMG1',
             'IM EMG2',
             'IM EMG3',
             'IM EMG4',
             'IM EMG5',
             'IM EMG6',
             'IM EMG7',
             'IM EMG8',
             'IM EMG9',
             'IM EMG10',
             'IM EMG11',
             'IM EMG12']
emg_labels_ref = ['Frame','Sub Frame',
             'EMG1',
             'EMG2',
             'EMG3',
             'EMG4',
             'EMG5',
             'EMG6',
             'EMG7',
             'EMG8',
             'EMG9',
             'EMG10',
             'EMG11',
             'EMG12']
marker_labels = ['Frame','Sub Frame',
              'RSHO_X','RSHO_Y','RSHO_Z',
              'RUPA_X','RUPA_Y','RUPA_Z',
              'RELB_X','RELB_Y','RELB_Z',
              'RM1_X','RM1_Y','RM1_Z',
             'RFRM_X','RFRM_Y','RFRM_Z',
             'WRM2_X','WRM2_Y','WRM2_Z',
             'RWRA_X','RWRA_Y','RWRA_Z',
             'RWRB_X','RWRB_Y','RWRB_Z',
             'RFIN_X','RFIN_Y','RFIN_Z']

In [61]:
path = 'Subjects/Nirmayee'
file = 'Bulb.csv'

In [62]:
file_s = file
file = '/marker_data/' + file

with open(path+'/'+file, "r") as f:
    lines = f.readlines()

In [56]:
emg_lines = []
for line in lines[5:]:
    if line=='\n':
        break
    emg_lines.append(line.split(','))
emg_df = pd.DataFrame(np.array(emg_lines),columns=lines[3].split(','))
emg_df = emg_df[emg_labels]
emg_df.columns = emg_labels_ref
#     print(emg_df)
#     print(emg_df.info())
#     print(emg_df.describe())
#     emg_df = emg_df.interpolate()
emg_df = emg_df.head(92000)
emg_df = emg_df[emg_df.columns].astype(float)
# duration = emg_df.shape[0]/2000

In [63]:
emg_df

,Frame,Sub Frame,EMG1,EMG2,EMG3,EMG4,EMG5,EMG6,EMG7,EMG8,EMG9,EMG10,EMG11,EMG12
0,1.0,0.0,-0.000001,-0.000040,-0.000050,0.000031,-0.000024,0.000018,-0.000023,0.000009,-0.000033,2.182160e-06,-0.000059,-0.000012
1,1.0,1.0,0.000004,-0.000035,-0.000047,0.000025,-0.000024,0.000020,-0.000019,0.000015,-0.000032,3.369060e-07,-0.000058,-0.000014
2,1.0,2.0,0.000006,-0.000032,-0.000045,0.000021,-0.000023,0.000021,-0.000019,0.000017,-0.000032,-7.880290e-07,-0.000057,-0.000016
3,1.0,3.0,0.000006,-0.000029,-0.000044,0.000017,-0.000021,0.000022,-0.000020,0.000016,-0.000032,-1.492560e-06,-0.000056,-0.000019
4,1.0,4.0,0.000007,-0.000024,-0.000044,0.000011,-0.000021,0.000022,-0.000021,0.000015,-0.000031,-2.613280e-06,-0.000055,-0.000022
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
53995,2700.0,15.0,0.000007,0.000054,-0.000041,0.000052,-0.000025,0.000020,-0.000020,0.000012,0.000001,2.109680e-04,-0.000062,0.000063
53996,2700.0,16.0,0.000004,0.000056,-0.000043,0.000037,-0.000023,0.000025,-0.000020,0.000011,-0.000006,1.979010e-04,-0.000063,0.000065
53997,2700.0,17.0,0.000003,0.000051,-0.000047,0.000022,-0.000020,0.000032,-0.000016,0.000008,-0.000017,1.325620e-04,-0.000065,0.000065
53998,2700.0,18.0,0.000004,0.000041,-0.000050,0.000003,-0.000018,0.000043,-0.000014,0.000004,-0.000028,5.532020e-05,-0.000066,0.000064


In [70]:
# Marker Data Frame
marker_lines = []
marker_line_start = None
for i in range(len(lines)):
    if lines[i]=='Trajectories\n':
        marker_line_start = i
        break
# marker_line_start = 92000+6
for line in lines[marker_line_start+5:]:
    if line=='\n':
        break
    marker_lines.append(line.split(','))
marker_df = pd.DataFrame(np.array(marker_lines),columns=marker_labels)
marker_df = marker_df[marker_df.columns].astype(float)
marker_df = marker_df.interpolate()

marker_df

TypeError: unsupported operand type(s) for +: 'NoneType' and 'int'

In [59]:
marker_line_start

54006

In [68]:
lines

['ï»¿Devices,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 '2000,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,\n',
 ',,Imported Delsys EMG 2.0.2 #1 - Sensor 1,Imported Delsys EMG 2.0.2 #1 - Sensor 2,Imported Delsys EMG 2.0.2 #1 - Sensor 3,Imported Delsys EMG 2.0.2 #1 - Sensor 4,Imported Delsys EMG 2.0.2 #1 - Sensor 5,Imported Delsys EMG 2.0.2 #1 - Sensor 6,Imported Delsys EMG 2.0.2 #1 - Sensor 7,Imported Delsys EMG 2.0.2 #1 - Sensor 8,Imported Delsys EMG 2.0.2 #1 - Sensor 9,Imported Delsys EMG 2.0.2 #1 - Sensor 10,Imported Delsys EMG 2.0.2 #1 - Sensor 11,Imported Delsys EMG 2.0.2 #1 - Sensor 12,Imported Delsys EMG 2.0.2 #1 - Sens